# DEFINICION DE LAS FUENTES DE HUMEDAD

En este ejercicio veremos como se calculan las fuentes de humedad que usaremos en los ejercios posteriores. Las fuentes se definen mediante una metodología euleriana a través de la divergencia en el flujo de humedad.

Los datos que se emplearan tanto es este ejercios como en los ejercicios siguientes están disponibles en el repositorio https://github.com/martavazquezdo/iao2021.git
Los archivos .dat pueden cargarse directamente desde el repositorio, sin embargo los netcdf deben descargarse y cargarse directamente como archivo local.

In [6]:
import numpy as np
from netCDF4 import Dataset
import math

In [16]:
datau=Dataset('/Users/martavazquezdominguez/Desktop/carpeta sin título/VIMF/east_flux_final_01_1980_2015.nc')
datav=Dataset('/Users/martavazquezdominguez/Desktop/carpeta sin título/VIMF/north_flux_final_01_1980_2015.nc')

In [17]:
lats = datau.variables['latitude'][:]
lons = datau.variables['longitude'][:]
times = datau.variables['time'][:]
uq= datau.variables['uq'][:][:][:]
vq= datav.variables['vq'][:][:][:]
datau.close()
datav.close()

Calculamos la media para las componentes de flujo

In [19]:
u=np.nanmean(uq,axis=0)
v=np.nanmean(vq,axis=0)

## CALCULO DIVERGENCIA

Datos para calcular las distancias

In [20]:
rt = 6371000. #Radio medio de la Tierra en metros
gr = math.pi/180.

Calculamos dx y dy

In [21]:
dfsln=np.diff(lons)
dfslt=np.diff(lats)

dx=np.zeros([180,359])
dy=np.zeros([180,359])

In [22]:
for ltr in range(0,180):
    for lnr in range(0,359):
        dx[ltr,lnr] = rt * math.cos (gr * lats[ltr]) * gr * dfsln[lnr] 


for ltr in range(0,180):
    for lnr in range(0,359):
        dy[ltr,lnr] = rt * gr * dfslt[ltr]

Esto es un factor de corrección necesario para el cálculo

In [23]:
corr=np.zeros([180,359])

for ltr in range(0,180):
    for lnr in range(0,359):
        corr[ltr,lnr]=v[ltr,lnr]* math.tan (gr * lats[ltr]) / rt

Calculo de las "derivadas" y divergencia

In [24]:
dfsuq=np.diff (u, axis=-1)
dfsvq=np.diff (v, axis=-2)

du_dx=np.divide(dfsuq[1:,:],dx)
dv_dy=np.divide(dfsvq[:,1:], dy)

divg = du_dx + dv_dy - corr  

El valor de la divergencia da valores falsos, para quitar estos valores eliminamos los datos con valor absoluto mayor que 1

In [33]:
divg=np.where(divg>1,np.nan,divg)
divg=np.where(divg<-1,np.nan,divg)

Los datos están en kg/m s. Debemos pasarlos a mm/dia

In [37]:
divg=divg*86400

Guardamos los datos

In [40]:
#SAVE RESULTS
            
datatotal = Dataset('divergence_01_.nc','w',format='NETCDF4_CLASSIC')
lat = datatotal.createDimension('lat', 180)
lon = datatotal.createDimension('lon', 359)
            
latitudes = datatotal.createVariable('latitude', np.float32,('lat',))
longitudes = datatotal.createVariable('longitude', np.float32,('lon',))
            
dataVIMF = datatotal.createVariable('DIVERGENCE', np.float32,('lat','lon'))
        
# Variable Attributes
            
latitudes.units = 'degree_north'
longitudes.units = 'degree_east'
dataVIMF.units = 'mm/day'
            
latitudes[:] = np.arange(90,-90,-1)
longitudes[:] = np.arange(0,359,1)
            
dataVIMF[:,:]=divg[:,:]

datatotal.close()

Las fuentes aqui definidas siguen la metodología establecida en el siguiente artículo: https://www.sciencedirect.com/science/article/abs/pii/S0309170819300818

Los umbrales utilizados para definir las fuentes de humedad son los siguientes:

<img src="Tabla_valores.png">

Los valores oceánicos se correponden con el percentil 50 de la divergencia del flujo

## Seleccionar el área de las fuentes Oceánicas

Calcular el umbral en mm/dia y eliminar los valores por debajo del umbral

Guardar los resultados